##Libraries

In [ ]:
!pip install transformers
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, AutoModelForQuestionAnswering
from tokenizers import BertWordPieceTokenizer
from tqdm.auto import tqdm
import os
import torch
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torchmetrics import Accuracy

##Making Tokenizer and prepare data

In [ ]:
df = pd.read_json('solution\\dataset\\train.json')
df_test = pd.read_json('solution\\dataset\\test.json')

In [ ]:
df_train, df_val = train_test_split(df, test_size=0.25)
label = 0
exist_text = 0
for i in range(len(df_train)):
    if df_train.iloc[i]['label'] == 'обеспечение исполнения контракта':
        label += 1
    else:
        label -= 1
    if len(df_train.iloc[i]['extracted_part']['text']) == 0:
        exist_text += 1
print(f'train len: {str(len(df_train))}')
print(f'train the same labels: {str(label)}\nnot existing pred text in {str(exist_text)}')

label = 0
exist_text = 0
for i in range(len(df_val)):
    if df_val.iloc[i]['label'] == 'обеспечение исполнения контракта':
        label += 1
    else:
        label -= 1
    if len(df_val.iloc[i]['extracted_part']['text']) == 0:
        exist_text += 1
print(f'val len: {str(len(df_val))}')
print(f'val the same labels: {str(label)}\nnot existing pred text in {str(exist_text)}')

train len: 1349
train the same labels: 127
not existing pred text in 0
val len: 450
val the same labels: 50
not existing pred text in 0


In [ ]:
df_train

,id,text,label,extracted_part
928,954631796,«УТВЕРЖДАЮ» Директор ГАУК РБ «НБ РБ» _________...,обеспечение исполнения контракта,{'text': ['обеспечение исполнения договора уст...
832,133688898,7 Договор поставки №______ г. Краснотурьинск «...,обеспечение исполнения контракта,{'text': ['Исполнитель представляет обеспечени...
1270,455554162,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,{'text': ['Размер обеспечения гарантийных обяз...
660,378442523,Акционерное общество «Российский концерн по пр...,обеспечение исполнения контракта,{'text': ['обеспечение исполнения договора в р...
276,882996752,ПРОЕКТ КОНТРАКТА КОНТРАКТ № ________ Поставка ...,обеспечение исполнения контракта,{'text': ['Поставщик до заключения Контракта п...
...,...,...,...,...
922,533236039,Государственное автономное учреждение здравоох...,обеспечение исполнения контракта,{'text': ['Поставщик при заключении Договора д...
688,286107658,«УТВЕРЖДАЮ» Директор филиала МАУДО ДООПЦ «Непт...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения догов...
1438,116840222,Приложение № 5 к извещению о проведении электр...,обеспечение гарантийных обязательств,{'text': ['по контракту Подрядчику надлежит пр...
503,741009080,ПРОЕКТ Гражданско-правовой договор бюджетного...,обеспечение исполнения контракта,{'text': ['Поставщик внес обеспечение исполнен...


In [ ]:
df_train.to_json('solution\\dataset\\mytrain.json')
df_val.to_json('solution\\dataset\\myval.json')

In [ ]:
text_data = []
file_count = 0

for i in tqdm(range(len(df_train))):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = df_train.iloc[i]['text'].replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5000:
        # once we hit the 5K mark, save to file
        with open(f'solution\\data4train_tokenizer\\text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'solution\\data4train_tokenizer\\text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/1799 [00:00<?, ?it/s]

In [ ]:
path = 'solution\\data4train_tokenizer'
files = [f'{path}/{f}' for f in os.listdir(path)]
files

['/content/drive/MyDrive/nlp_test_task_2023/train_test_tokenize_in_txt/text_0.txt']

In [ ]:
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=False
)
# and train
tokenizer.train(files=files, vocab_size=30000, min_frequency=2,
                limit_alphabet=1000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

In [ ]:
tokenizer.save_model('solution\\tokenizer')

['/content/drive/MyDrive/nlp_test_task_2023/tokenizer/vocab.txt']

##Help funcs

In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in tqdm(range(len(answers))):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'][0]))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'][0]))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'][0] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
def train_epoch(epoch, model, loader, optimizer, device, accuracy_metric, loss_fn):
    model.train()
    losses = 0
    value_accuracy = 0
    for i, batch in enumerate(loader):
        input_batch = {'input_ids': batch['input_ids'].to(device),
                       'attention_mask': batch['attention_mask'].to(device),
                       'start_positions': batch['start_positions'].type(torch.LongTensor).to(device),
                       'end_positions': batch['end_positions'].type(torch.LongTensor).to(device)}

        start_positions = input_batch['start_positions']
        end_positions = input_batch['end_positions']

        outputs = model(input_batch)
        
        optimizer.zero_grad()
        loss = (loss_fn(outputs['start_logits'], start_positions) + loss_fn(outputs['end_logits'], end_positions)) / 2
        value_accuracy += (accuracy_metric(outputs['start_pos'], start_positions) + accuracy_metric(outputs['end_pos'], end_positions)) / 2
        
        loss.backward()
        optimizer.step()
        losses += loss.item()

    with open(f'solution\\train_losses\\epoch_{str(epoch)}.txt', '+a') as f:
        f.write(str(losses / len(loader))+'\n')
    return {'loss': losses/len(loader),
            'accuracy': value_accuracy.item()/len(loader)}

In [ ]:
def evaluate(epoch, model, loader, device, accuracy_metric, loss_fn):
    model.eval()
    losses = 0
    value_accuracy = 0

    for i, batch in enumerate(loader):
        input_batch = {'input_ids': batch['input_ids'].to(device),
                       'attention_mask': batch['attention_mask'].to(device),
                       'start_positions': batch['start_positions'].type(torch.LongTensor).to(device),
                       'end_positions': batch['end_positions'].type(torch.LongTensor).to(device)}

        start_positions = input_batch['start_positions']
        end_positions = input_batch['end_positions']

        with torch.no_grad():
            outputs = model(input_batch)
            
            loss = (loss_fn(outputs['start_logits'], start_positions) + loss_fn(outputs['end_logits'], end_positions)) / 2
            losses += loss.item()

            value_accuracy += (accuracy_metric(outputs['start_pos'], start_positions) + accuracy_metric(outputs['end_pos'], end_positions)) / 2
            
    with open(f'solution\\val_losses\\epoch_{epoch}.txt', '+a') as f:
        f.write(str(losses / len(loader))+'\n')
    return {'loss': losses/len(loader),
            'accuracy': value_accuracy.item()/len(loader)}

In [ ]:
def get_test_predict_of_words_on_one_sample(model, context, label, tokenizer):
    model.eval()
    test_data = tokenizer(context, label,
                     truncation=True, 
                     padding='max_length',
                     max_length=512, 
                     return_tensors='pt',
                     return_offsets_mapping=True)
    offset_mapping = test_data.pop('offset_mapping')
    with torch.no_grad():
        output = model.get_predict(test_data)
    data = offset_mapping[0][output['start_pos'].item():output['end_pos'].item()]
    if len(data) != 0:
        start_pos_in_context = data[0][0].item()
        end_pos_in_context = data[-1][-1].item() + 1
    else:
        start_pos_in_context = 0
        end_pos_in_context = 0
    return {'text': [context[start_pos_in_context:end_pos_in_context]],
            'answer_start': [start_pos_in_context],
            'answer_end': [end_pos_in_context]}

##CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        output_data = {}
        for key, val in self.encodings.items():
            if not isinstance(val[idx], torch.Tensor):
                output_data[key] = torch.tensor(val[idx])
            else:
                output_data[key] = val[idx]
        return output_data

    def __len__(self):
        return len(self.encodings.input_ids)

##making Dataset

In [ ]:
df_train = pd.read_json('solution\\dataset\\mytrain.json')
df_train = df_train.reset_index()
# Drop rows from dataframe, because this is a break data
df_train = df_train.drop([df_train.index[41], df_train.index[925], df_train.index[926]])

df_val = pd.read_json('solution\\dataset\\myval.json')
df_val = df_val.reset_index()


tokenizer = BertTokenizerFast.from_pretrained('solution\\tokenizer')
train = tokenizer(list(df_train['text']), list(df_train['label']),
                  truncation=True, padding='max_length',
                  max_length=512, return_tensors='pt')
add_token_positions(train, list(df_train['extracted_part']))
print(f'train len: {len(df_train)}\ntrain keys: {train.keys()}')
val = tokenizer(list(df_val['text']), list(df_val['label']),
                  truncation=True, padding='max_length',
                  max_length=512, return_tensors='pt')
add_token_positions(val, list(df_val['extracted_part']))
print(f'val len: {len(df_val)}\nval keys: {val.keys()}')

# build datasets for both our training data
train_dataset = CustomDataset(train)
train_loader = DataLoader(train_dataset,
                          batch_size=16,
                          shuffle=True)
val_dataset = CustomDataset(val)
val_loader = DataLoader(val_dataset,
                        batch_size=16,
                        shuffle=True)

  0%|          | 0/1346 [00:00<?, ?it/s]

train len: 1346
train keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


  0%|          | 0/450 [00:00<?, ?it/s]

val len: 450
val keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [ ]:
df_train.head()

,index,id,text,label,extracted_part
0,1780,390815281,ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1,369,836142280,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
2,1569,648286513,Министерство культуры Российской Федерации Фед...,обеспечение гарантийных обязательств,{'text': ['к участникам закупки устанавливаетс...
3,1553,856330320,Согласовано протоколом Тендерной комиссии АО М...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
4,1318,854906389,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,{'text': ['Размер обеспечения гарантийных обяз...


##Model

In [ ]:
class BertQA(torch.nn.Module):
    def __init__(self):
        super(BertQA, self).__init__()
        self.model = AutoModelForQuestionAnswering.from_pretrained('bert-base-cased')
        self.log_softmax = torch.nn.LogSoftmax(dim=1)

    def forward(self, x):
        input_ids = x['input_ids']
        attention_mask = x['attention_mask']
        start_positions = x['start_positions']
        end_positions = x['end_positions']
        x = self.model(input_ids, attention_mask=attention_mask,
                       start_positions=start_positions,
                       end_positions=end_positions)
        output = {}
        start_logits = self.log_softmax(x.start_logits)
        end_logits = self.log_softmax(x.end_logits)
        output['start_pos'] = start_logits.argmax(-1)
        output['end_pos'] = end_logits.argmax(-1)
        output['start_logits'] = start_logits
        output['end_logits'] = end_logits
        return output
    
    def get_predict(self, x):
        x = self.model(**x)
        output = {}
        start_logits = self.log_softmax(x.start_logits)
        end_logits = self.log_softmax(x.end_logits)
        output['start_pos'] = start_logits.argmax(-1)
        output['end_pos'] = end_logits.argmax(-1)
        return output

##ModelParams

In [ ]:
num_train_epochs = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = BertQA().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
accuracy_metric = Accuracy(task='multiclass', num_classes=tokenizer.vocab_size).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
device

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

device(type='cuda', index=0)

##Train

In [ ]:
for epoch in range(num_train_epochs):
    print(f'Training epoch: {str(epoch)}...')
    data_of_train = train_epoch(epoch=epoch,
                                model=model,
                                loader=train_loader,
                                optimizer=optimizer,
                                device=device,
                                accuracy_metric=accuracy_metric,
                                loss_fn=loss_fn)
    print(f'loss: {str(data_of_train["loss"])}, accuracy: {str(data_of_train["accuracy"])}')
    print(f'Evaluating epoch: {str(epoch)}...')
    data_of_evaluate = evaluate(epoch=epoch,
                                model=model,
                                loader=train_loader,
                                device=device,
                                accuracy_metric=accuracy_metric,
                                loss_fn=loss_fn)
    print(f'loss: {str(data_of_evaluate["loss"])}, accuracy: {str(data_of_evaluate["accuracy"])}')
    torch.save(model.state_dict(), f'solution\\models\\{str(epoch)}epoch.pt')

Training epoch: 0...
loss: 2.778326697910533, accuracy: 0.506985294117647
Evaluating epoch: 0...
loss: 1.0780002818388097, accuracy: 0.7647058823529411
Training epoch: 1...
loss: 1.0004291180302114, accuracy: 0.7867647058823529
Evaluating epoch: 1...
loss: 0.5996308892965316, accuracy: 0.8599264705882353
Training epoch: 2...
loss: 0.6638265406384187, accuracy: 0.8441176470588235
Evaluating epoch: 2...
loss: 0.42495555334231433, accuracy: 0.9007352941176471
Training epoch: 3...
loss: 0.4989057135932586, accuracy: 0.8775735294117647
Evaluating epoch: 3...
loss: 0.29349079904666936, accuracy: 0.9283088235294118
Training epoch: 4...
loss: 0.3519029545433381, accuracy: 0.9102941176470588
Evaluating epoch: 4...
loss: 0.17963699027895927, accuracy: 0.9577205882352942
Training epoch: 5...
loss: 0.262068184004987, accuracy: 0.93125
Evaluating epoch: 5...
loss: 0.15188591451767613, accuracy: 0.9613970588235294
Training epoch: 6...
loss: 0.22753326811772936, accuracy: 0.9404411764705882
Evaluatin

##Test

In [ ]:
model = BertQA()
checkpoint = torch.load('solution\\models\\9epoch.pt', map_location='cpu')
model.load_state_dict(checkpoint)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('solution\\tokenizer')

In [ ]:
df_test = pd.read_json('solution\\dataset\\test.json')

In [ ]:
df_test.head()

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта


In [ ]:
data_for_add = []
for i in range(len(df_test)):
    sample = get_test_predict_of_words_on_one_sample(model=model, 
                                                     context=df_test.iloc[i]['text'],
                                                     label=df_test.iloc[i]['label'],
                                                     tokenizer=tokenizer)
    data_for_add.append(sample)

df_test['extracted_part'] = data_for_add

In [ ]:
df_test.head()

,id,text,label,extracted_part
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения Контр...
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,{'text': ['Размер обеспечение исполнения контр...


In [ ]:
df_test.to_csv('/content/drive/MyDrive/nlp_test_task_2023/predictions.csv', index=False)